In [1]:
using Arrow
using CSV
using DataFrames
using Statistics

In [2]:
url = ("https://archive.ics.uci.edu/ml/machine-learning-databases/"
       * "auto-mpg/auto-mpg.data-original")
download(url, "auto.txt")
readlines("auto.txt")

406-element Vector{String}:
 "18.0   8.   307.0      130.0      3504.      12.0   70.  1.\t\"chevrolet chevelle malibu\""
 "15.0   8.   350.0      165.0      3693.      11.5   70.  1.\t\"buick skylark 320\""
 "18.0   8.   318.0      150.0      3436.      11.0   70.  1.\t\"plymouth satellite\""
 "16.0   8.   304.0      150.0      3433.      12.0   70.  1.\t\"amc rebel sst\""
 "17.0   8.   302.0      140.0      3449.      10.5   70.  1.\t\"ford torino\""
 "15.0   8.   429.0      198.0      4341.      10.0   70.  1.\t\"ford galaxie 500\""
 "14.0   8.   454.0      220.0      4354.       9.0   70.  1.\t\"chevrolet impala\""
 "14.0   8.   440.0      215.0      4312.       8.5   70.  1.\t\"plymouth fury iii\""
 "14.0   8.   455.0      225.0      4425.      10.0   70.  1.\t\"pontiac catalina\""
 "15.0   8.   390.0      190.0      3850.       8.5   70.  1.\t\"amc ambassador dpl\""
 "NA     4.   133.0      115.0      3090.      17.5   70.  2.\t\"citroen ds-21 pallas\""
 "NA     8.   350.0      1

In [3]:
rawstr = read("auto.txt", String);
strnotab = replace(rawstr, '\t' => ' ')
io = IOBuffer(strnotab)

IOBuffer(data=UInt8[...], readable=true, writable=false, seekable=true, append=false, size=32149, maxsize=Inf, ptr=1, mark=-1)

In [10]:
fields = [:mpg, :cylinders, :displacement, :horsepower, :weight, 
          :acceleration, :year, :origin, :name]
df1 = CSV.File(
    io,
    delim=' ',
    ignorerepeated=true,
    header=fields,
    missingstring="NA"
) |> DataFrame

,mpg,cylinders,displacement,horsepower,weight,acceleration,year,origin,name
,Float64?,Float64,Float64,Float64?,Float64,Float64,Float64,Float64,String
1,18.0,8.0,307.0,130.0,3504.0,12.0,70.0,1.0,chevrolet chevelle malibu
2,15.0,8.0,350.0,165.0,3693.0,11.5,70.0,1.0,buick skylark 320
3,18.0,8.0,318.0,150.0,3436.0,11.0,70.0,1.0,plymouth satellite
4,16.0,8.0,304.0,150.0,3433.0,12.0,70.0,1.0,amc rebel sst
5,17.0,8.0,302.0,140.0,3449.0,10.5,70.0,1.0,ford torino
6,15.0,8.0,429.0,198.0,4341.0,10.0,70.0,1.0,ford galaxie 500
7,14.0,8.0,454.0,220.0,4354.0,9.0,70.0,1.0,chevrolet impala
8,14.0,8.0,440.0,215.0,4312.0,8.5,70.0,1.0,plymouth fury iii
9,14.0,8.0,455.0,225.0,4425.0,10.0,70.0,1.0,pontiac catalina


In [7]:
ENV["COLUMNS"], ENV["LINES"] = 200, 10
df1

,mpg,cylinders,displacement,horsepower,weight,acceleration,year,origin,name
,Float64?,Float64,Float64,Float64?,Float64,Float64,Float64,Float64,String
1,18.0,8.0,307.0,130.0,3504.0,12.0,70.0,1.0,chevrolet chevelle malibu
2,15.0,8.0,350.0,165.0,3693.0,11.5,70.0,1.0,buick skylark 320
3,18.0,8.0,318.0,150.0,3436.0,11.0,70.0,1.0,plymouth satellite
4,16.0,8.0,304.0,150.0,3433.0,12.0,70.0,1.0,amc rebel sst
5,17.0,8.0,302.0,140.0,3449.0,10.5,70.0,1.0,ford torino
6,15.0,8.0,429.0,198.0,4341.0,10.0,70.0,1.0,ford galaxie 500
7,14.0,8.0,454.0,220.0,4354.0,9.0,70.0,1.0,chevrolet impala
8,14.0,8.0,440.0,215.0,4312.0,8.5,70.0,1.0,plymouth fury iii
9,14.0,8.0,455.0,225.0,4425.0,10.0,70.0,1.0,pontiac catalina


In [8]:
dfraw = CSV.File("auto.txt", header=[:metrics, :name]) |> DataFrame

,metrics,name
,String,String
1,18.0 8. 307.0 130.0 3504. 12.0 70. 1.,chevrolet chevelle malibu
2,15.0 8. 350.0 165.0 3693. 11.5 70. 1.,buick skylark 320
3,18.0 8. 318.0 150.0 3436. 11.0 70. 1.,plymouth satellite
4,16.0 8. 304.0 150.0 3433. 12.0 70. 1.,amc rebel sst
5,17.0 8. 302.0 140.0 3449. 10.5 70. 1.,ford torino
6,15.0 8. 429.0 198.0 4341. 10.0 70. 1.,ford galaxie 500
7,14.0 8. 454.0 220.0 4354. 9.0 70. 1.,chevrolet impala
8,14.0 8. 440.0 215.0 4312. 8.5 70. 1.,plymouth fury iii
9,14.0 8. 455.0 225.0 4425. 10.0 70. 1.,pontiac catalina


In [9]:
strmetrics = split.(dfraw.metrics)

406-element Vector{Vector{SubString{String}}}:
 ["18.0", "8.", "307.0", "130.0", "3504.", "12.0", "70.", "1."]
 ["15.0", "8.", "350.0", "165.0", "3693.", "11.5", "70.", "1."]
 ["18.0", "8.", "318.0", "150.0", "3436.", "11.0", "70.", "1."]
 ⋮
 ["28.0", "4.", "120.0", "79.00", "2625.", "18.6", "82.", "1."]
 ["31.0", "4.", "119.0", "82.00", "2720.", "19.4", "82.", "1."]

In [16]:
df12 = DataFrame([col => Float64[] for col in fields[1:(end - 1)]])

,mpg,cylinders,displacement,horsepower,weight,acceleration,year,origin
,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64


In [17]:
allowmissing!(df12, [:mpg, :horsepower])

,mpg,cylinders,displacement,horsepower,weight,acceleration,year,origin
,Float64?,Float64,Float64,Float64?,Float64,Float64,Float64,Float64


In [18]:
for row in strmetrics
    push!(df12, [v == "NA" ? missing : parse(Float64, v) for v in row])
end

In [19]:
df12

,mpg,cylinders,displacement,horsepower,weight,acceleration,year,origin
,Float64?,Float64,Float64,Float64?,Float64,Float64,Float64,Float64
1,18.0,8.0,307.0,130.0,3504.0,12.0,70.0,1.0
2,15.0,8.0,350.0,165.0,3693.0,11.5,70.0,1.0
3,18.0,8.0,318.0,150.0,3436.0,11.0,70.0,1.0
4,16.0,8.0,304.0,150.0,3433.0,12.0,70.0,1.0
5,17.0,8.0,302.0,140.0,3449.0,10.5,70.0,1.0
6,15.0,8.0,429.0,198.0,4341.0,10.0,70.0,1.0
7,14.0,8.0,454.0,220.0,4354.0,9.0,70.0,1.0
8,14.0,8.0,440.0,215.0,4312.0,8.5,70.0,1.0
9,14.0,8.0,455.0,225.0,4425.0,10.0,70.0,1.0


In [21]:
df12.name = dfraw.name
df12

,mpg,cylinders,displacement,horsepower,weight,acceleration,year,origin,name
,Float64?,Float64,Float64,Float64?,Float64,Float64,Float64,Float64,String
1,18.0,8.0,307.0,130.0,3504.0,12.0,70.0,1.0,chevrolet chevelle malibu
2,15.0,8.0,350.0,165.0,3693.0,11.5,70.0,1.0,buick skylark 320
3,18.0,8.0,318.0,150.0,3436.0,11.0,70.0,1.0,plymouth satellite
4,16.0,8.0,304.0,150.0,3433.0,12.0,70.0,1.0,amc rebel sst
5,17.0,8.0,302.0,140.0,3449.0,10.5,70.0,1.0,ford torino
6,15.0,8.0,429.0,198.0,4341.0,10.0,70.0,1.0,ford galaxie 500
7,14.0,8.0,454.0,220.0,4354.0,9.0,70.0,1.0,chevrolet impala
8,14.0,8.0,440.0,215.0,4312.0,8.5,70.0,1.0,plymouth fury iii
9,14.0,8.0,455.0,225.0,4425.0,10.0,70.0,1.0,pontiac catalina


In [22]:
df12.name === dfraw.name #!

true